In [ ]:
# if load on colab
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/ManufacturingQA/') # Your folder
!pip install -r /content/drive/MyDrive/Archive/Main/requirements.txt

In [ ]:
import argparse
import os
import pandas as pd
import sys

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../2codes'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# load local function
from utils import generate_response, get_prompt_by_type, data_processor, save_results_to_folder

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--QAType", default=2, type=int, help="Which Type of Question to be asked, we have 1.MCQ, 2.ResponseQ, 3.MathQ, 4.ImageQ")
    parser.add_argument("--verbose", default=True, type=bool, help="Print the output or not")
    parser.add_argument("--evidence", default=False, type=bool, help="Print and Plot the source of the answer")
    parser.add_argument("--top_k", default=3, type=int, help="Top k answer to be generated")
    parser.add_argument("--interactive", default=False, type=bool, help="User can input the question or use default")
    parser.add_argument("--model", default="gpt-4o", type=str, help="The model to be used, currently supports gpt-4o")
    parser.add_argument("--document", default="1Book/3Book_txt_images", type=str, help="The document (Book) to be used")

    # This line filters out the jupyter notebook arguments
    if sys.argv and len(sys.argv) > 1:
        return parser.parse_args(args=[])
    else:
        return parser.parse_args()

# Parsing arguments
args = parse_args()
print(args)
# Not printing the output
args.api_key=os.getenv("OPENAI_API_KEY")
import os
os.chdir("../../") # change to main directory
!pwd

In [5]:
# list of all question folders
RQfolder = "2QuestionsData/1RQ"
MCQfolder = "2QuestionsData/2MCQ"
ImgQfolder = "2QuestionsData/3ImgQ/"
MathQfolder = "2QuestionsData/4MathQ/"

In [6]:
def load_csv_files_from_folder(folder_path):
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    # Load all CSVs into dataframes
    dfs = [pd.read_csv(os.path.join(folder_path, f), header=0) for f in csv_files]
    
    if dfs:
        args.questions_df = pd.concat(dfs, ignore_index=True)  # Concatenate into one dataframe
        
        return args
    else:
        raise ValueError(f"No CSV files found in folder: {folder_path}")

In [7]:
# Easy handle for quick testing
args.QAType = 4
if args.QAType == 1:
    load_csv_files_from_folder(MCQfolder)    
elif args.QAType == 2:
    load_csv_files_from_folder(RQfolder)
elif args.QAType == 3:
    load_csv_files_from_folder(ImgQfolder)
elif args.QAType == 4:
    load_csv_files_from_folder(MathQfolder)
else:
    raise ValueError("Invalid QAType. Please specify a valid QAType from 1 to 4.")


Give a virtualization for data and use small testset for demo

In [8]:
args.questions_df = args.questions_df[:3]
args.questions_df

,chapter,question_number,Question,Answer
0,23,23.01,23.01 Flank wear data were collected in a seri...,Solution: (a) and (b) Student exercises. For p...
1,23,23.02,23.02 Solve Problem 23.1 except that the tool ...,Solution: (a) and (b) Student exercises. For p...
2,23,23.03,23.03 A series of turning tests were conducted...,"Solution: (a) Using the graph, at 350 ft/min ..."


In [ ]:
os.environ["OPENAI_API_KEY"] = args.api_key
GPT_results = []
GPT_cot_results = []
prompt_data = get_prompt_by_type(args)
for question in args.questions_df['Question']:
    # GPT (no prompt design)
    GPT_result = generate_response(args, query=question, image_paths="")
    
    # GPT_cot (with prompt design)
    prompt = prompt_data.format(context_str="", query_str=question)
    GPT_cot_result = generate_response(args, query=prompt, image_paths="")
    
    # Append results to args
    GPT_results.append(GPT_result)
    GPT_cot_results.append(GPT_cot_result)

# Define a dictionary to loop over results and process/save them
results_mapping = {
    "GPT_results": GPT_results,
    "GPT_cot_results": GPT_cot_results,
}

# Process and save each result type
for result_type, result_list in results_mapping.items():
    if result_list:  # Only process if the result list is not empty
        results_df = data_processor(result_type,result_list, args)
        save_results_to_folder(results_df, args, result_type)
